Config 

In [ ]:
import json
import time
from datetime import datetime
from pathlib import Path
from typing import Optional
import logging

class ResultLogger:
    """
    A utility class for saving verification results to individual member JSON files.
    Each member gets their own result file that is overwritten on each run.
    """
    
    def __init__(self, results_directory: str = "results"):
        """
        Initialize the ResultLogger.
        
        Args:
            results_directory (str): Directory where result files will be saved
        """
        self.results_dir = Path(results_directory)
        self.results_dir.mkdir(exist_ok=True)
        
    def save_result(self, 
                   member_name: str,
                   student_id: str, 
                   score: float,
                   runtime: float,
                   timestamp: Optional[str] = None) -> bool:
        """
        Save verification result to a member-specific JSON file.
        
        Args:
            member_name (str): Name of the member/algorithm (e.g., "Member1", "Member2")
            student_id (str): Student ID from QR code
            score (float): Verification confidence score (0.0 to 1.0)
            runtime (float): Total verification time in seconds
            timestamp (str, optional): ISO timestamp. If None, uses current time.
            
        Returns:
            bool: True if saved successfully, False otherwise
        """
        try:
            # Generate timestamp if not provided
            if timestamp is None:
                timestamp = datetime.now().isoformat()
            
            # Create result data
            result_data = {
                "member": member_name,
                "student_id": student_id,
                "score": round(score, 4),  # Round to 4 decimal places
                "runtime": round(runtime, 4),  # Round to 4 decimal places
                "timestamp": timestamp
            }
            
            # Generate filename based on member name
            filename = f"{member_name.lower()}_results.json"
            filepath = self.results_dir / filename
            
            if filepath.exists():
                with open(filepath, 'r', encoding='utf-8') as f:
                    try:
                        records = json.load(f)
                        if not isinstance(records, list):
                            records = [records]  # handle old single-record format
                    except json.JSONDecodeError:
                        records = []
            else:
                records = []


            # Save to file (overwrite existing)
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(result_data, f, indent=2, ensure_ascii=False)
            
            print(f"✓ Result saved for {member_name}: Score={score:.3f}, Runtime={runtime:.3f}s")
            return True
            
        except Exception as e:
            print(f"✗ Failed to save result for {member_name}: {e}")
            return False

# Initialize the logger
result_logger = ResultLogger()
print("ResultLogger initialized successfully!")

ResultLogger initialized successfully!


In [15]:
# Cell 1: Imports and Configuration
import cv2
from pyzbar.pyzbar import decode
import os
import sys
import json
from datetime import datetime
import numpy as np
import hashlib
import time
from typing import Tuple, List, Optional, Dict, Any
import logging
from dataclasses import dataclass
from pathlib import Path

# GUI Specific Imports
import customtkinter as ctk
from PIL import Image, ImageTk
from tkinter import messagebox
from skimage.feature import local_binary_pattern

import pyttsx3
import threading
from tkinter import Toplevel
from queue import Queue, Empty


@dataclass
class Config:
    FACE_DB_PATH: str = "student_faces"
    LOG_FILE: str = "verification_log.json"
    STUDENT_DATA_FILE: str = "student_database.json"
    SUPPORTED_EXTENSIONS: List[str] = None
    
    # Camera settings
    CAMERA_WIDTH: int = 640
    CAMERA_HEIGHT: int = 480
    CAMERA_FPS: int = 25
    
    # SIFT parameters
    SIFT_FEATURES: int = 4000
    SIFT_CONTRAST_THRESHOLD: float = 0.015
    SIFT_EDGE_THRESHOLD: int = 12
    
    # Verification thresholds
    MATCH_THRESHOLD: float = 0.7
    MIN_KEYPOINTS: int = 15
    MIN_GOOD_MATCHES: int = 10
    REQUIRED_CONSECUTIVE: int = 3
    LOWE_RATIO: float = 0.7
    
    # Quality thresholds
    MIN_FACE_SIZE: int = 80
    PREFERRED_FACE_SIZE: int = 120
    MIN_PHOTOS_REQUIRED: int = 5
    DEFAULT_PHOTOS_COUNT: int = 12

    # NEW: Lighting adaptation parameters
    BRIGHT_THRESHOLD: float = 180.0
    DARK_THRESHOLD: float = 70.0
    OVEREXPOSE_RATIO_THRESHOLD: float = 0.1
    UNDEREXPOSE_RATIO_THRESHOLD: float = 0.1

    # NEW: Timeout settings
    VERIFICATION_TIMEOUT: int = 30  # seconds
    QR_SCAN_TIMEOUT: int = 15      # seconds
    
    def __post_init__(self):
        if self.SUPPORTED_EXTENSIONS is None:
            self.SUPPORTED_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.bmp']

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('system.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [16]:
class StudentSystem:
    def __init__(self, config: Config = None):
        self.config = config or Config()
        self._setup_directories()
        self.load_student_data()
        self.sift = self._create_sift_detector()
        self.matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
        )
        self.verification_stats = {
            'total_attempts': 0, 'successful_verifications': 0,
            'failed_verifications': 0, 'avg_verification_time': 0.0
        }
        
        
        logger.info("Student Verification System backend initialized")

    def _setup_directories(self):
        Path(self.config.FACE_DB_PATH).mkdir(exist_ok=True)

    def _create_sift_detector(self):
        return cv2.SIFT_create(
            nfeatures=self.config.SIFT_FEATURES,
            contrastThreshold=self.config.SIFT_CONTRAST_THRESHOLD,
            edgeThreshold=self.config.SIFT_EDGE_THRESHOLD
        )

    def load_student_data(self):
        try:
            if Path(self.config.STUDENT_DATA_FILE).exists():
                with open(self.config.STUDENT_DATA_FILE, 'r', encoding='utf-8') as f:
                    self.student_data = json.load(f)
                logger.info(f"Loaded data for {len(self.student_data)} students")
            else:
                self.student_data = {}
                logger.info("No existing student data found, starting fresh")
        except (json.JSONDecodeError, IOError) as e:
            logger.error(f"Error loading student data: {e}")
            self.student_data = {}

    def get_student_info(self, student_id: str) -> Optional[Dict[str, Any]]:
        return self.student_data.get(student_id)

    def log_verification(self, student_id: str, name: str, program: str, success: bool, 
                         timestamp: str, score: float = 0.0, verification_time: float = 0.0):
        log_entry = {
            "student_id": student_id, "name": name, "program": program, "success": success,
            "timestamp": timestamp, "score": score, "verification_time": verification_time,
            "session_id": hashlib.md5(f"{timestamp}{student_id}".encode()).hexdigest()[:8]
        }
        try:
            logs = []
            if Path(self.config.LOG_FILE).exists():
                with open(self.config.LOG_FILE, 'r', encoding='utf-8') as f:
                    logs = json.load(f)
            logs.append(log_entry)
            if len(logs) > 1000:
                logs = logs[-1000:]
            with open(self.config.LOG_FILE, 'w', encoding='utf-8') as f:
                json.dump(logs, f, indent=2, ensure_ascii=False)
            status = "SUCCESS" if success else "FAILED"
            logger.info(f"Verification logged: {name} - {status} (Score: {score:.3f})")
        except IOError as e:
            logger.error(f"Error logging verification: {e}")

    def view_logs(self) -> str:
        if not Path(self.config.LOG_FILE).exists():
            return "📝 No verification logs found."
        try:
            with open(self.config.LOG_FILE, 'r', encoding='utf-8') as f:
                logs = json.load(f)
            if not logs:
                return "📝 No verification logs found."
            
            header = f"{'Timestamp':<20} {'Student ID':<12} {'Name':<25} {'Status':<10} {'Score':<8} {'Time(s)':<8}\n"
            divider = "=" * 90 + "\n"
            report = "📊 Recent Verification Logs\n" + divider + header + divider
            
            for log in reversed(logs[-50:]): # Show newest logs first
                # --- ROBUSTNESS FIX START ---
                try:
                    # Attempt to format the timestamp
                    timestamp = datetime.fromisoformat(log['timestamp']).strftime("%Y-%m-%d %H:%M:%S")
                except (TypeError, ValueError):
                    # If it fails, use a placeholder and continue
                    timestamp = "INVALID TIMESTAMP"
                # --- ROBUSTNESS FIX END ---
    
                status = "✅ SUCCESS" if log.get('success', False) else "❌ FAILED"
                score = f"{log.get('score', 0)*100:.1f}%"
                time_taken = f"{log.get('verification_time', 0):.1f}"
                name = log.get('name', 'N/A')[:24]
                student_id = log.get('student_id', 'N/A')
                
                report += f"{timestamp:<20} {student_id:<12} {name:<25} {status:<10} {score:<8} {time_taken:<8}\n"
    
            total = len(logs)
            successful = sum(1 for log in logs if log.get('success'))
            success_rate = (successful / total * 100) if total > 0 else 0
            report += divider + f"📈 Summary: {successful}/{total} successful ({success_rate:.1f}% success rate)"
            return report
        except Exception as e:
            logger.error(f"Error reading logs: {e}")
            return f"❌ Error reading logs: {e}"

    def validate_qr_data(self, qr_data: str) -> Tuple[Optional[str], Optional[str], Optional[str]]:
        student_id = qr_data.strip()
        student_info = self.get_student_info(student_id)
        if student_info:
            name = student_info.get("name", "N/A")
            program = student_info.get("program", "N/A")
            logger.info(f"QR ID '{student_id}' found in database for student: {name}")
            return student_id, name, program
        else:
            logger.warning(f"QR ID '{student_id}' not found in the student database.")
            return None, None, None

    def load_registered_faces(self, student_id: str) -> List[np.ndarray]:
        folder = Path(self.config.FACE_DB_PATH) / student_id
        if not folder.is_dir(): return []
        images = []
        for file_path in folder.glob("*"):
            if file_path.suffix.lower() in self.config.SUPPORTED_EXTENSIONS:
                try:
                    img = cv2.imread(str(file_path))
                    if img is not None and img.size > 0:
                        images.append(img)
                except Exception as e:
                    logger.warning(f"Failed to load image {file_path}: {e}")
        logger.info(f"Loaded {len(images)} face images for student {student_id}")
        return images

    def preprocess_face(self, face_img: np.ndarray) -> np.ndarray:
        if len(face_img.shape) == 3:
            gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        else:
            gray = face_img.copy()

        resized = cv2.resize(gray, (220, 220), interpolation=cv2.INTER_CUBIC)
    
        # Adaptive lighting analysis
        mean_brightness = np.mean(resized)
        std_brightness = np.std(resized)
        overexposed_pixels = np.sum(resized >= 240) / resized.size
        underexposed_pixels = np.sum(resized <= 20) / resized.size
    
        # Adaptive brightness correction
        if mean_brightness > 180 or overexposed_pixels > 0.1:
            # Very bright/overexposed - reduce brightness and enhance shadows
            alpha, beta = 0.7, -25
            resized = cv2.convertScaleAbs(resized, alpha=alpha, beta=beta)
            gamma = 1.3
            resized = np.power(resized / 255.0, gamma) * 255.0
            resized = resized.astype(np.uint8)
        elif mean_brightness < 70 or underexposed_pixels > 0.1:
            # Dark/underexposed - increase brightness
            alpha, beta = 1.3, 25
            resized = cv2.convertScaleAbs(resized, alpha=alpha, beta=beta)
            gamma = 0.7
            resized = np.power(resized / 255.0, gamma) * 255.0
            resized = resized.astype(np.uint8)
    
        # Adaptive CLAHE based on contrast
        if std_brightness < 30:  # Low contrast
            clip_limit, tile_size = 3.0, (8, 8)
        elif std_brightness > 60:  # High contrast
            clip_limit, tile_size = 1.5, (12, 12)
        else:  # Normal contrast
            clip_limit, tile_size = 2.0, (8, 8)
    
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_size)
        enhanced = clahe.apply(resized)
    
        # Create a sharpening kernel
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        sharpened = cv2.filter2D(enhanced, -1, kernel)
    
        # Blend original enhanced with sharpened (80% enhanced, 20% sharpened)
        blended = cv2.addWeighted(enhanced, 0.8, sharpened, 0.2, 0)
    
        # Final normalization
        normalized = cv2.normalize(blended, None, 0, 255, cv2.NORM_MINMAX)
    
        return normalized.astype(np.uint8)

    
    def calculate_match_score(
        self, matches: List, kp1, kp2, img1_shape: Tuple, img2_shape: Tuple
    ) -> float:
        """
        A simplified scoring function based primarily on the number of geometrically
        verified matches (inliers).
        """
        # --- 1) Basic check for minimum matches ---
        if len(matches) < self.config.MIN_GOOD_MATCHES:
            return 0.0

        # --- 2) Geometric verification (RANSAC) ---
        # This is the most critical step, so we keep it.
        src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

        M, mask = cv2.estimateAffinePartial2D(
            src_pts, dst_pts, method=cv2.RANSAC,
            ransacReprojThreshold=5.0, maxIters=1000, confidence=0.98
        )
        
        # If RANSAC fails or finds no model, there's no match.
        if mask is None:
            return 0.0
            
        inlier_count = int(np.sum(mask.ravel()))

        # --- 3) Simple Score Calculation ---
        # The score is now just a measure of how many inliers we found,
        # scaled against a "target" number of inliers.
        target_inliers = 15  # We aim for at least 15 solid matches

        # Clip the score between 0.0 and 1.0
        final_score = min(inlier_count / target_inliers, 1.0)
        
        return float(final_score)

    def export_verification_report(self) -> str: # MODIFIED: Returns status string for GUI
        if not Path(self.config.LOG_FILE).exists():
            return "No verification logs found to export."
        try:
            with open(self.config.LOG_FILE, 'r', encoding='utf-8') as f: logs = json.load(f)
            if not logs: return "No verification data to export."
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            report_file = f"verification_report_{timestamp}.json"
            
            total = len(logs)
            successful = sum(1 for log in logs if log['success'])
            success_rate = (successful / total * 100) if total > 0 else 0
            
            report = {
                'generated_at': datetime.now().isoformat(),
                'summary': {
                    'total_attempts': total,
                    'successful_verifications': successful,
                    'success_rate_percentage': round(success_rate, 2),
                },
                'recent_logs': logs[-50:]
            }
            with open(report_file, 'w', encoding='utf-8') as f:
                json.dump(report, f, indent=2, ensure_ascii=False)
            return f"Report successfully exported to {report_file}"
        except Exception as e:
            logger.error(f"Error generating report: {e}")
            return f"Failed to generate report: {e}"


In [17]:
class RobustTTS:
    def __init__(self):
        self.speech_queue = Queue()
        self.worker_thread = threading.Thread(target=self._worker_loop, daemon=True)
        self.worker_thread.start()
        logger.info("RobustTTS worker thread started.")

    def _worker_loop(self):
        """
        Processes speech requests from the queue.
        Each request gets a new, temporary TTS engine.
        """
        while True:
            try:
                # Wait indefinitely for a new speech task
                text = self.speech_queue.get()

                # A special signal to terminate the thread
                if text is None:
                    break

                # --- The Core Fix ---
                # Create a new engine instance for this specific text.
                # This is the key to avoiding state-related errors.
                engine = pyttsx3.init(driverName='sapi5')

                # Configure the engine (optional but recommended)
                voices = engine.getProperty('voices')
                if voices:
                    for voice in voices:
                        if 'female' in voice.name.lower() or 'zira' in voice.name.lower():
                            engine.setProperty('voice', voice.id)
                            break
                engine.setProperty('rate', 155)
                engine.setProperty('volume', 0.9)

                # Speak the text
                engine.say(text)
                engine.runAndWait()

                # Cleanly shut down the temporary engine
                del engine

            except Empty:
                # This part is unlikely to be reached with a blocking .get()
                # but is good practice.
                continue
            except Exception as e:
                # Log any errors that occur during TTS synthesis
                logger.error(f"TTS worker loop error: {e}")

    def speak(self, text: str):
        """
        Adds a text string to the speech queue to be spoken.
        This method is non-blocking.
        """
        if text and text.strip():
            # Clear any pending speech requests to say the newest one
            while not self.speech_queue.empty():
                try:
                    self.speech_queue.get_nowait()
                except Empty:
                    break
            # Add the new text to the queue
            self.speech_queue.put(text.strip())

    def cleanup(self):
        """
        Sends a signal to the worker thread to terminate gracefully.
        Call this when the application is closing.
        """
        logger.info("Cleaning up TTS system...")
        self.speech_queue.put(None)  # Sentinel value to stop the worker
        self.worker_thread.join(timeout=2.0) # Wait for thread to finish
        logger.info("TTS system cleaned up.")

In [18]:
class App(ctk.CTk):
    def __init__(self, student_system: StudentSystem):
        super().__init__()
        self.system = student_system
        
        self.tts_system = RobustTTS()

        self.member_name = "Member1"

        self.title("🎓 Student Verification System")
        self.geometry("1100x720")
        ctk.set_appearance_mode("Dark")
        ctk.set_default_color_theme("blue")
        
        # --- App State ---
        self.cap = None
        self.verification_active = False
        self.current_student_id = None
        self.current_student_name = None
        self.current_student_program = None
        self.reference_data = []
        self.consecutive_matches = 0
        self.best_score = 0.0
        self.last_known_face = None
        self.frame_count = 0
        self.verification_step = "idle" # States: idle, qr_scan, face_verify

        # NEW: Add timeout tracking variables
        self.verification_start_time = None
        self.qr_scan_start_time = None
        self.max_verification_time = 30  # seconds
        self.max_qr_scan_time = 15      # seconds
        
        # --- Layout ---
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)

        # --- Sidebar Frame ---
        self.sidebar_frame = ctk.CTkFrame(self, width=200, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(5, weight=1)
        
        self.logo_label = ctk.CTkLabel(self.sidebar_frame, text="Main Menu", font=ctk.CTkFont(size=20, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        
        self.verify_button = ctk.CTkButton(self.sidebar_frame, text="Verify Student", command=self.verify_student_frame_event)
        self.verify_button.grid(row=1, column=0, padx=20, pady=10)

        self.logs_button = ctk.CTkButton(self.sidebar_frame, text="View Logs", command=self.view_logs_frame_event)
        self.logs_button.grid(row=2, column=0, padx=20, pady=10)

        self.export_button = ctk.CTkButton(self.sidebar_frame, text="Export Report", command=self.export_report_event)
        self.export_button.grid(row=3, column=0, padx=20, pady=10)

        self.exit_button = ctk.CTkButton(self.sidebar_frame, text="Exit", fg_color="transparent", border_width=2, text_color=("gray10", "#DCE4EE"), command=self.on_closing)
        self.exit_button.grid(row=6, column=0, padx=20, pady=20, sticky="s")
        
        # --- Verification Frame ---
        self.verify_frame = ctk.CTkFrame(self, corner_radius=0, fg_color="transparent")
        self.verify_frame.grid_columnconfigure(0, weight=1)
        
        self.camera_label = ctk.CTkLabel(self.verify_frame, text="")
        self.camera_label.grid(row=0, column=0, padx=10, pady=10)

        self.status_label = ctk.CTkLabel(self.verify_frame, text="Welcome! Click 'Start Verification' to begin.", font=ctk.CTkFont(size=16))
        self.status_label.grid(row=1, column=0, padx=10, pady=5)
        
        self.info_label = ctk.CTkLabel(self.verify_frame, text="", font=ctk.CTkFont(size=14))
        self.info_label.grid(row=2, column=0, padx=10, pady=5)
        
        self.progress_bar = ctk.CTkProgressBar(self.verify_frame, width=300)
        self.progress_bar.set(0) # Start it at 0
        self.progress_bar.grid(row=3, column=0, padx=10, pady=5)

        self.start_button = ctk.CTkButton(self.verify_frame, text="Start Verification", command=self.start_verification_process)
        self.start_button.grid(row=4, column=0, padx=10, pady=10)

        # --- Logs Frame ---
        self.logs_frame = ctk.CTkFrame(self, corner_radius=0, fg_color="transparent")
        self.logs_frame.grid_columnconfigure(0, weight=1)
        self.logs_frame.grid_rowconfigure(0, weight=1)
        self.log_textbox = ctk.CTkTextbox(self.logs_frame, width=700, font=("Courier New", 12))
        self.log_textbox.grid(row=0, column=0, padx=20, pady=20, sticky="nsew")
        


        # --- Initial Setup ---
        self.select_frame_by_name("verify")
        self.protocol("WM_DELETE_WINDOW", self.on_closing)

    def setup_tts_engine(self):
        """Configure the text-to-speech engine"""
        try:
            # Set properties for better voice quality
            voices = self.tts_engine.getProperty('voices')
            if voices:
                # Try to use a female voice if available, otherwise use default
                for voice in voices:
                    if 'female' in voice.name.lower() or 'zira' in voice.name.lower():
                        self.tts_engine.setProperty('voice', voice.id)
                        break
            
            # Set speech rate (words per minute)
            self.tts_engine.setProperty('rate', 150)  # Slower for clarity
            
            # Set volume (0.0 to 1.0)
            self.tts_engine.setProperty('volume', 0.8)
            
        except Exception as e:
            logger.warning(f"TTS engine setup warning: {e}")

    def speak_text(self, text):
        self.tts_system.speak(text)

    def show_student_details_screen(self, student_id: str, verification_score: float):
        """
        Display a detailed screen with student information and voice announcement
        
        Args:
            student_id (str): The student ID
            verification_score (float): The final verification score
        """
        student_info = self.system.get_student_info(student_id)
        if not student_info:
            logger.error(f"No student info found for ID: {student_id}")
            return
        
        # Create a new window for student details
        details_window = Toplevel(self)
        details_window.title("✅ Verification Successful")
        details_window.geometry("600x500")
        details_window.configure(bg='#1a1a1a')
        details_window.resizable(False, False)
        
        # Make window modal and center it
        details_window.transient(self)
        details_window.grab_set()
        
        # Center the window
        details_window.update_idletasks()
        x = (details_window.winfo_screenwidth() // 2) - (600 // 2)
        y = (details_window.winfo_screenheight() // 2) - (500 // 2)
        details_window.geometry(f"1200x700+{x}+{y}")
        
        # Create main frame
        main_frame = ctk.CTkFrame(details_window, fg_color="transparent")
        main_frame.pack(fill="both", expand=True, padx=20, pady=20)
        
        # Success header
        success_label = ctk.CTkLabel(
            main_frame, 
            text="🎉 VERIFICATION SUCCESSFUL 🎉",
            font=ctk.CTkFont(size=24, weight="bold"),
            text_color="#00ff00"
        )
        success_label.pack(pady=(0, 20))
        
        # Student photo - load from card_photos folder
        photo_frame = ctk.CTkFrame(main_frame, width=180, height=200, fg_color="#2b2b2b")
        photo_frame.pack(pady=(0, 20))
        photo_frame.pack_propagate(False)  # Maintain frame size
        
        # Try to load student photo from card_photos folder
        photo_loaded = False
        try:
            # Look for photo in card_photos folder with student ID as filename
            card_photos_folder = Path("card_photos")
            
            # Try different extensions
            for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                photo_path = card_photos_folder / f"{student_id}{ext}"
                if photo_path.exists():
                    img = cv2.imread(str(photo_path))
                    if img is not None:
                        # Convert BGR to RGB
                        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        
                        # Resize image to fit frame (170x190 with padding)
                        img_pil = Image.fromarray(img_rgb)
                        img_pil = img_pil.resize((170, 190), Image.Resampling.LANCZOS)
                        
                        # Create CTk image
                        photo_ctk = ctk.CTkImage(
                            light_image=img_pil, 
                            dark_image=img_pil, 
                            size=(170, 190)
                        )
                        
                        # Display the photo
                        photo_label = ctk.CTkLabel(
                            photo_frame, 
                            image=photo_ctk,
                            text=""
                        )
                        photo_label.pack(expand=True, padx=5, pady=5)
                        photo_loaded = True
                        logger.info(f"Loaded student photo: {photo_path}")
                        break
                        
        except Exception as e:
            logger.warning(f"Could not load student photo: {e}")
        
        # If no photo loaded, show placeholder
        if not photo_loaded:
            photo_label = ctk.CTkLabel(
                photo_frame, 
                text="👤\nNo Photo\nAvailable",
                font=ctk.CTkFont(size=14),
                text_color="#cccccc"
            )
            photo_label.pack(expand=True)
            logger.info(f"No photo found for student {student_id} in card_photos folder")
        
        # Student details frame
        details_frame = ctk.CTkFrame(main_frame, fg_color="#2b2b2b")
        details_frame.pack(fill="x", pady=(0, 20))
        
        # Extract student information
        name = student_info.get("name", "N/A")
        program = student_info.get("program", "N/A")
        batch = student_info.get("batch", "N/A")
        academic_performance = student_info.get("academic_performance", "N/A")
        cgpa = student_info.get("cgpa", "N/A")
        
        # Create detail labels
        details_data = [
            ("👤 Name:", name),
            ("📚 Program:", program),
            ("🎓 Batch:", f"20{batch}" if batch.isdigit() else batch),
            ("⭐ Academic Performance:", academic_performance),
            ("📊 CGPA:", cgpa),
            ("🔍 Verification Score:", f"{verification_score*100:.1f}%")
        ]
        
        for i, (label, value) in enumerate(details_data):
            detail_frame = ctk.CTkFrame(details_frame, fg_color="transparent")
            detail_frame.pack(fill="x", padx=20, pady=5)
            
            label_widget = ctk.CTkLabel(
                detail_frame,
                text=label,
                font=ctk.CTkFont(size=14, weight="bold"),
                anchor="w"
            )
            label_widget.pack(side="left", fill="x", expand=True)
            
            value_widget = ctk.CTkLabel(
                detail_frame,
                text=str(value),
                font=ctk.CTkFont(size=14),
                text_color="#00bfff",
                anchor="e"
            )
            value_widget.pack(side="right")
        
        # Buttons frame
        button_frame = ctk.CTkFrame(main_frame, fg_color="transparent")
        button_frame.pack(fill="x", pady=(10, 0))
        
        # Close button
        close_button = ctk.CTkButton(
            button_frame,
            text="✓ Close",
            command=details_window.destroy,
            fg_color="#2196F3",
            hover_color="#1976D2"
        )
        close_button.pack(side="right")
        
        # Auto-close timer (optional)
        def auto_close():
            try:
                if details_window.winfo_exists():
                    details_window.destroy()
            except:
                pass
        
        # Auto close after 15 seconds
        details_window.after(15000, auto_close)
        
        # Make initial voice announcement
        self.announce_student_details(name, program, batch, academic_performance)
        
        return details_window

    def announce_student_details(self, name: str, program: str, batch: str, academic_performance: str):
        """Create and speak the announcement text"""
        # Create a natural announcement
        announcement_parts = [
            f"Verification successful.",
            f"Welcome, {name}.",
            f"Program: {program}.",
            f"Batch 20{batch}." if batch.isdigit() else f"Batch {batch}.",
            f"Academic performance: {academic_performance}.",
            "Access granted. Have a great day!"
        ]
        
        announcement_text = " ".join(announcement_parts)
        
        # Log the announcement
        logger.info(f"Voice announcement: {announcement_text}")
        
        # Speak the announcement
        self.speak_text(announcement_text)

    def select_frame_by_name(self, name):
        # Hide all frames
        self.verify_frame.grid_remove()
        self.logs_frame.grid_remove()
        
        # Show the selected frame
        if name == "verify":
            self.verify_frame.grid(row=0, column=1, sticky="nsew")
        elif name == "logs":
            self.logs_frame.grid(row=0, column=1, sticky="nsew")
            
    def verify_student_frame_event(self):
        self.select_frame_by_name("verify")

    def view_logs_frame_event(self):
        self.select_frame_by_name("logs")
        logs = self.system.view_logs()
        self.log_textbox.configure(state="normal")
        self.log_textbox.delete("1.0", "end")
        self.log_textbox.insert("1.0", logs)
        self.log_textbox.configure(state="disabled")
        

    def export_report_event(self):
        message = self.system.export_verification_report()
        messagebox.showinfo("Export Report", message)

    def check_timeouts(self):
        """Check if any timeout conditions are met"""
        current_time = time.time()

        # Check QR scan timeout
        if self.verification_step == "qr_scan" and self.qr_scan_start_time:
            qr_elapsed = current_time - self.qr_scan_start_time
            if qr_elapsed > self.max_qr_scan_time:
                self.handle_timeout("qr_scan")
                return True

        # Check face verification timeout
        elif self.verification_step == "face_verify" and self.verification_start_time:
            verification_elapsed = current_time - self.verification_start_time
            if verification_elapsed > self.max_verification_time:
                self.handle_timeout("face_verify")
                return True
    
        return False
    
    def handle_timeout(self, timeout_type):
        """Handle different types of timeouts"""
        if timeout_type == "qr_scan":
            self.speak_text("QR code scan timeout. Please try again.")
            messagebox.showwarning("Timeout", "QR code scanning timed out. Please try again.")
        elif timeout_type == "face_verify":
            self.speak_text("Face verification timeout. Access denied.")
            messagebox.showwarning("Timeout", "Face verification timed out. Please try again.")

        # Log the failed verification
        if hasattr(self, 'current_student_id') and self.current_student_id:
            timestamp = datetime.now().isoformat()
            self.system.log_verification(
                self.current_student_id or "UNKNOWN",
                self.current_student_name or "Unknown User",
                self.current_student_program or "Unknown Program",
                False,  # success = False
                timestamp,
                score=self.best_score,
                verification_time=time.time() - self.verification_start_time if self.verification_start_time else 0
            )

        # Finalize as failed verification
        self.finalize_verification(False)
    
    def start_verification_process(self):
        if self.verification_active: return
        
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Camera Error", "Cannot open camera.")
            return

        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.system.config.CAMERA_WIDTH)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.system.config.CAMERA_HEIGHT)
        
        self.verification_active = True
        self.verification_step = "qr_scan"

        self.verification_start_time = time.time()
        self.qr_scan_start_time = time.time()

        self.start_button.configure(state="disabled", text="Verification in Progress...")
        self.verification_start_time = time.time()
        
        self.update_frame()

    def stop_verification_process(self):
        self.verification_active = False
        self.verification_step = "idle"
        if self.cap:
            self.cap.release()
            self.cap = None
        self.start_button.configure(state="normal", text="Start Verification")
        # Reset state for next run
        self.current_student_id = None
        self.reference_data = []
        self.consecutive_matches = 0
        self.best_score = 0.0
        self.last_known_face = None
        self.frame_count = 0
        self.progress_bar.set(0)

    def display_timeout_info(self):
        """Display remaining time information"""
        current_time = time.time()

        if self.verification_step == "qr_scan" and self.qr_scan_start_time:
            elapsed = current_time - self.qr_scan_start_time
            remaining = max(0, self.max_qr_scan_time - elapsed)
            self.info_label.configure(
                text=f"Position QR code in view. Time remaining: {remaining:.0f}s"
            )

        elif self.verification_step == "face_verify" and self.verification_start_time:
            elapsed = current_time - self.verification_start_time
            remaining = max(0, self.max_verification_time - elapsed)
            self.info_label.configure(
                text=f"Face verification. Time: {remaining:.0f}s"
            )

    def update_frame(self):
        if not self.verification_active:
            return

        if self.check_timeouts():
            return 
    
        ret, frame = self.cap.read()
        if not ret:
            self.after(15, self.update_frame)
            return
        
        frame = cv2.flip(frame, 1)

        self.display_timeout_info()
        
        if self.verification_step == "qr_scan":
            self.handle_qr_scan(frame)
        elif self.verification_step == "face_verify":
            self.handle_face_verify(frame)
        
        # Convert frame for display
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        ctk_img = ctk.CTkImage(light_image=img, dark_image=img, size=(self.system.config.CAMERA_WIDTH, self.system.config.CAMERA_HEIGHT))
        self.camera_label.configure(image=ctk_img)
        self.camera_label.image = ctk_img

        self.after(15, self.update_frame) # Schedule next update

    def handle_qr_scan(self, frame):
        self.status_label.configure(text="📱 Step 1: Scan Student QR Code")
        self.info_label.configure(text="Position the QR code in the camera view.")

        decoded_objects = decode(frame)
        for obj in decoded_objects:
            qr_data = obj.data.decode("utf-8")
            student_id, name, program = self.system.validate_qr_data(qr_data)
            
            if all([student_id, name, program]):
                self.current_student_id = student_id
                self.current_student_name = name
                self.current_student_program = program
                
                # Draw on frame
                points = obj.polygon
                hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
                cv2.polylines(frame, [np.int32(hull)], True, (0, 255, 0), 3)
                
                # Transition to face verification
                self.prepare_face_verification()
                return # Exit scan loop for this frame

    def prepare_face_verification(self):
        self.status_label.configure(text="⚙️ Preparing face verification...", text_color="yellow")
        self.info_label.configure(text=f"Student: {self.current_student_name} ({self.current_student_program})")
        
        registered_faces = self.system.load_registered_faces(self.current_student_id)
        if not registered_faces:
            messagebox.showerror("Error", f"No face data found for {self.current_student_name}.")
            self.stop_verification_process()
            return
            
        def auto_adjust_brightness(image, target_brightness=120):
            mean_val = np.mean(image)
            if mean_val > 160:  # Too bright
                scale = target_brightness / (mean_val * 1.2)  # More aggressive reduction
            elif mean_val < 80:  # Too dark
                scale = target_brightness / (mean_val * 0.8)  # More aggressive increase
            else:  # Normal range
                scale = target_brightness / mean_val
    
            # Apply scaling with saturation protection
            adjusted = cv2.convertScaleAbs(image, alpha=scale, beta=0)
    
            # Prevent over-saturation
            adjusted = np.clip(adjusted, 0, 255)
    
            return adjusted

        for ref_img in registered_faces:
            processed_ref = self.system.preprocess_face(ref_img)
            processed_ref = auto_adjust_brightness(processed_ref)
            kp, des = self.system.sift.detectAndCompute(processed_ref, None)
            if des is not None and len(kp) >= self.system.config.MIN_KEYPOINTS:
                self.reference_data.append({'kp': kp, 'des': des, 'shape': processed_ref.shape})

        if not self.reference_data:
            messagebox.showerror("Error", f"Could not process reference images for {self.current_student_name}.")
            self.stop_verification_process()
            return

        self.verification_step = "face_verify"
        self.verification_start_time = time.time()

    def handle_face_verify(self, frame):
        threshold_percent = self.system.config.MATCH_THRESHOLD * 100
        self.status_label.configure(text=f"📸 Step 2: Position Your Face Clearly (Threshold: {threshold_percent:.0f}%)")
        self.frame_count += 1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
        if self.frame_count % 2 == 0:
            faces = self.system.face_cascade.detectMultiScale(
                gray, scaleFactor=1.1, minNeighbors=5, 
                minSize=(self.system.config.MIN_FACE_SIZE, self.system.config.MIN_FACE_SIZE)
            )
            if len(faces) > 0:
                faces = sorted(faces, key=lambda f: f[2] * f[3], reverse=True)
                self.last_known_face = faces[0]
            else:
                self.last_known_face = None
    
        if self.last_known_face is not None:
            x, y, w, h = self.last_known_face
            
            def auto_adjust_brightness(image, target_brightness=110):
                mean_val = np.mean(image)
                if mean_val == 0: 
                    return image
                scale = target_brightness / mean_val
                return cv2.convertScaleAbs(image, alpha=scale, beta=0)
    
            face_img = gray[y:y+h, x:x+w]
            processed_face = self.system.preprocess_face(face_img)

            if np.mean(processed_face) > 200 or np.mean(processed_face) < 50:
                processed_face = cv2.equalizeHist(processed_face)

            processed_face = auto_adjust_brightness(processed_face)
            
            kp1, des1 = self.system.sift.detectAndCompute(processed_face, None)
            
            final_score = 0.0
        
            if des1 is not None and len(kp1) >= self.system.config.MIN_KEYPOINTS:
                # 1. Collect all valid scores for this frame in a list
                frame_scores = []
                for ref in self.reference_data:
                    kp2, des2, ref_shape = ref['kp'], ref['des'], ref['shape']
                    try:
                        raw_matches = self.system.matcher.knnMatch(des1, des2, k=2)
                        good_matches = []
                        # Ensure raw_matches is not empty and items have 2 elements
                        if raw_matches and len(raw_matches[0]) == 2:
                            for m, n in raw_matches:
                                if m.distance < self.system.config.LOWE_RATIO * n.distance:
                                    good_matches.append(m)
    
                        if len(good_matches) >= self.system.config.MIN_GOOD_MATCHES:
                            score = self.system.calculate_match_score(
                                good_matches, kp1, kp2, processed_face.shape, ref_shape
                            )
                            # Add the calculated score to our list for this frame
                            frame_scores.append(score)
    
                    except Exception:
                        # Safely skip this reference image if an error occurs
                        continue
                    
                # 2. Process the collected scores
                if frame_scores:
                    # Sort the scores from highest to lowest
                    frame_scores.sort(reverse=True)
                    
                    # Take the top 3 scores (or fewer if less than 3 are available)
                    top_scores = frame_scores[:3]
                    
                    # Calculate the average of these top scores
                    final_score = sum(top_scores) / len(top_scores)
            
            self.best_score = max(self.best_score, final_score)
    
            self.progress_bar.set(final_score)

            if final_score > self.system.config.MATCH_THRESHOLD:
                self.consecutive_matches += 1
                status_text = f"MATCHING... {self.consecutive_matches}/{self.system.config.REQUIRED_CONSECUTIVE}"
                status_color = "#32CD32"
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            else:
                self.consecutive_matches = 0
                status_text = "NO MATCH"
                status_color = "red"
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
    
            score_text = f"{final_score*100:.1f}%"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.7
            thickness = 2
            text_size = cv2.getTextSize(score_text, font, font_scale, thickness)[0]
            text_x = x
            text_y = max(y - 10, 0)  # slightly above the box
            cv2.putText(frame, score_text, (text_x, text_y), font, font_scale, (0, 255, 0) if final_score > self.system.config.MATCH_THRESHOLD else (0,0,255), thickness)

            self.info_label.configure(
                text=f"Score: {final_score*100:.1f}% | Best: {self.best_score*100:.1f}% | Status: {status_text}",
                text_color=status_color
            )
    
            if self.consecutive_matches >= self.system.config.REQUIRED_CONSECUTIVE:
                self.finalize_verification(True)
        else:
            self.info_label.configure(text="No face detected. Please ensure good lighting.", text_color="orange")
            self.progress_bar.set(0)

    def finalize_verification(self, success):

        end_time = time.time()
        runtime = end_time - self.verification_start_time if self.verification_start_time else 0.0

        # NEW: Capture the final score BEFORE resetting the state.
        final_score = self.best_score 
        
        timestamp = datetime.now().isoformat()
        student_id = self.current_student_id
        student_name = self.current_student_name
        student_program = self.current_student_program
        self.system.log_verification(
            student_id, student_name,
            student_program, success, timestamp,    
            score=final_score  # MODIFIED: Use the captured score for logging.
        )
        
        # Now, it's safe to stop and reset the process.
        self.stop_verification_process()
        
        if success:
            
            self.status_label.configure(text="✅ ACCESS GRANTED", text_color="green")
            # MODIFIED: Use the captured score for the final message.
            self.info_label.configure(text=f"Welcome, {student_name}! Final Score: {final_score*100:.1f}%", text_color="green")
            self.show_student_details_screen(student_id, final_score)
        else:
            self.status_label.configure(text="❌ ACCESS DENIED", text_color="red")
            self.info_label.configure(text=f"Face not recognized. Best score: {final_score*100:.1f}%", text_color="red")
            messagebox.showerror("Failure", "Face verification failed.")
            
        try:
            result_logger.save_result(
                member_name=self.member_name,
                student_id=student_id,
                score=final_score,
                runtime=runtime
            )
        except Exception as e:
            print(f"Failed to log result: {e}")
            
        # Reset labels to default after a delay
        self.after(5000, self.reset_labels)
    
    def reset_labels(self):
        self.status_label.configure(text="Welcome! Click 'Start Verification' to begin.", text_color=("white", "black"))
        self.progress_bar.set(0)
        self.info_label.configure(text="")
        

    def on_closing(self):
        # Stop verification if still active
        if self.verification_active:
            self.stop_verification_process()
    
        # Release camera if still open
        if self.cap and self.cap.isOpened():
            self.cap.release()
            self.cap = None
    
        # Stop TTS engine safely
        try:
            if hasattr(self, 'tts_system'):
                self.tts_system.cleanup()
        except Exception as e:
            print(f"TTS cleanup failed: {e}")
    
        # Close Tkinter
        self.quit()     # stop mainloop
        self.destroy()  # destroy window
    
        sys.exit(0)     # graceful exit

In [19]:
if __name__ == "__main__":
    try:
        config = Config()
        backend_system = StudentSystem(config)
        app = App(student_system=backend_system)
        app.mainloop()
    except KeyboardInterrupt:
        print("\n👋 System interrupted by user. Goodbye!")
    except Exception as e:
        logger.critical(f"Critical system error on startup: {e}")
        print(f"💥 Critical error: {e}")

2025-08-28 10:41:37,554 - INFO - Loaded data for 1 students
2025-08-28 10:41:37,603 - INFO - Student Verification System backend initialized
2025-08-28 10:41:37,749 - INFO - RobustTTS worker thread started.
2025-08-28 10:42:39,046 - INFO - Verification logged: None - FAILED (Score: 0.000)


✓ Result saved for Member1: Score=0.000, Runtime=48.560s


2025-08-28 10:42:48,635 - INFO - QR ID 'S001' found in database for student: Leong Kah Yung
2025-08-28 10:42:48,663 - INFO - Loaded 30 face images for student S001
2025-08-28 10:43:24,548 - INFO - Verification logged: Leong Kah Yung - FAILED (Score: 0.000)
2025-08-28 10:43:24,566 - INFO - Verification logged: Leong Kah Yung - FAILED (Score: 0.000)


✓ Result saved for Member1: Score=0.000, Runtime=35.409s


2025-08-28 10:43:36,469 - INFO - QR ID 'S001' found in database for student: Leong Kah Yung
2025-08-28 10:43:36,482 - INFO - Loaded 30 face images for student S001
2025-08-28 10:43:43,232 - INFO - Verification logged: Leong Kah Yung - SUCCESS (Score: 1.000)
2025-08-28 10:43:43,540 - INFO - Loaded student photo: card_photos\S001.jpg
2025-08-28 10:43:43,567 - INFO - Voice announcement: Verification successful. Welcome, Leong Kah Yung. Program: RSW. Batch 2022. Academic performance: First Class. Access granted. Have a great day!


✓ Result saved for Member1: Score=1.000, Runtime=6.338s


2025-08-28 10:44:01,875 - INFO - Cleaning up TTS system...
2025-08-28 10:44:01,877 - INFO - TTS system cleaned up.


SystemExit: 0